In [1]:
import pandas as pd

In [2]:
score = pd.read_csv("score.tsv", delimiter="\t")

In [10]:
import subprocess
from subprocess import Popen, PIPE, STDOUT
from pathlib import Path

def make_script(video_path, force=False):
    
    keypoint_path = Path(video_path).with_suffix(".json")
    if not force and keypoint_path.is_file():
        print("Keypoint Exist - ", keypoint_path)
        return

    cmd = [
        "python",
        "demo/body3d_two_stage_video_demo.py",
        "demo/mmdetection_cfg/faster_rcnn_r50_fpn_coco.py",
        "https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth",

        "configs/body/2d_kpt_sview_rgb_img/topdown_heatmap/coco/hrnet_w48_coco_256x192.py",
        "https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w48_coco_256x192-b9e0b3ab_20200708.pth",
        "configs/body/3d_kpt_sview_rgb_vid/video_pose_lift/h36m/videopose3d_h36m_243frames_fullconv_supervised_cpn_ft.py",
        "https://download.openmmlab.com/mmpose/body3d/videopose/videopose_h36m_243frames_fullconv_supervised_cpn_ft-88f5abbb_20210527.pth",
         
        "--video-path", video_path,
        "--keypoint-path", keypoint_path,
        "--out-video-root", "vis_results",
        "--num-instances", "1",
        "--rebase-keypoint-height",
    ]
    
    return cmd

In [11]:
processes = []

In [13]:

import os, re, mmcv

test_cnt = 10000
cmd_d = {}
basedir = '/mnt/c/Users/USER/Dropbox/머신러닝_수강생영상 (2)/'
fps_error = []
for i, r in score.iterrows():
    title = r['title'].strip()
    filename = r['file name']
    score1 = r['Skill']
    score2 = r['Expression']

    title = re.sub(r"[\*\?]", "", title)
    filename = re.sub(r"[\*\?]", "", filename)

    if filename != "1986_STATION X 0_Wow Thing":
        continue

    found = False
    dir1 = Path(basedir) / title
    for file in os.listdir(dir1):
        if file.startswith(filename) and not file.endswith("json"):
            filename = file
            found = True

    if not found:
        print(i, "Video Not Found", dir1 / filename)
        continue

    x_path = dir1 / filename
    
    found = False
    dir2 = dir1 / "강사님 영상"
    for file in os.listdir(dir2):
        if not file.startswith(".") and not file.endswith("json"):
            y_path = dir2 / file
            found = True
    
    if not found:
        print(i, "Teacher Video Not Found", dir2)
        continue
    vid_x = mmcv.VideoReader(str(x_path))
    vid_y = mmcv.VideoReader(str(y_path))
    if abs(vid_x.fps - vid_y.fps) > 1:
        print(i, "FPS Problem ", vid_x.fps, vid_y.fps, "|", filename)
        fps_error.append(filename)
        continue

    cmd = make_script(x_path, force=True)
    if cmd is not None:
        cmd_d[x_path] = cmd
        
        
    cmd = make_script(y_path, force=True)
    if cmd is not None:
        cmd_d[y_path] = cmd
    test_cnt -= 1
    if test_cnt == 0:
        break

In [14]:
keys = list(cmd_d.keys())
len(keys)

2

In [15]:
import time

current_running = []
max_run = 2
while(len(keys) > 0):
    to_remove = []
    for p in current_running:
        if p.poll() is not None:
            to_remove.append(p)
    print(len(to_remove))
    if len(to_remove):
        for p in to_remove:
            current_running.remove(p)
    else:
        time.sleep(5)
    while(len(current_running) < max_run and len(keys) > 0):
        k = keys.pop()
        p = Popen(cmd_d[k])
        current_running.append(p)
for p in current_running:
    p.wait()

0
Keypoint Exist
Stage 1: 2D pose detection.
Initializing model...
Keypoint Exist
Stage 1: 2D pose detection.
Initializing model...
load checkpoint from http path: https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth
load checkpoint from http path: https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth
load checkpoint from http path: https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w48_coco_256x192-b9e0b3ab_20200708.pth
load checkpoint from http path: https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w48_coco_256x192-b9e0b3ab_20200708.pth
Running 2D pose detection inference...
[                                                  ] 0/1514, elapsed: 0s, ETA:Running 2D pose detection inference...
[                                                  ] 0/1560, elapsed: 0s, ETA:

KeyboardInterrupt: 

In [80]:
for p in current_running:
    p.kill()

In [135]:
for p in current_running:
    p.wait()

[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 1176/1176, 5.4 task/s, elapsed: 217s, ETA:     0s
Stage 2: 2D-to-3D pose lifting.
Initializing model...
load checkpoint from http path: https://download.openmmlab.com/mmpose/body3d/videopose/videopose_h36m_243frames_fullconv_supervised_cpn_ft-88f5abbb_20210527.pth
Running 2D-to-3D pose lifting inference...
[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 1176/1176, 12.2 task/s, elapsed: 96s, ETA:     0s
